<a href="https://colab.research.google.com/github/Suvarnaph/hello-world/blob/master/ImageLoader_V9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#Import statements
from google.colab import drive
drive.mount('/content/drive/')

from __future__ import absolute_import, division, print_function
from pathlib import Path
import re
import os
from os import getcwd
from os.path import exists, isdir, isfile, join
import shutil
import numpy as np
import pandas as pd
import glob
import cv2

%matplotlib inline
from matplotlib import pyplot as plt

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
class Image_Loader:
  
    def __init__(self, root_path, source_dir_name, size, img_read_format, img_type = ('*.jpg', '*.png')):
        self.project_path = root_path 
        self.root = Path(self.project_path)
        self.source_directory = Path.joinpath(self.root, source_dir_name) 
        self.scaled_size = size 
        self.formatRead = img_read_format
        self.img_format_type = img_type
        self.x_list=[]
        self.y_list=[]
        self.outpath = Path.joinpath(self.source_directory,"Processed_Image","XR_SHOULDER",source_dir_name) 
        Path(self.outpath).mkdir(parents=True, exist_ok=True)
        
    #Labelling Images
    def label_img(self,img):
      word_label = os.path.basename(os.path.dirname(img))
      #print(word_label)
      if "positive" in word_label: 
        return 1
      elif "negative" in word_label:
        return 0
      
      
      
    #Image_processing method
    def image_read_resize(self, file, scaled_size, formatRead = cv2.IMREAD_GRAYSCALE):
      formatRead = cv2.IMREAD_GRAYSCALE
      OrigImage = cv2.imread(file,formatRead)
      #resizing image
      width = scaled_size
      height = scaled_size
      dim = (width, height)
      resizedImg = cv2.resize(OrigImage, dim)
      return resizedImg
    
    
    #Convert Image to list and numpy array; Study - XR_SHOULDER:
    def image_to_list_numpy(self):
      for study_category in self.source_directory.iterdir():
        if(os.path.isdir(study_category)):
          if study_category == (Path.joinpath(self.source_directory,"XR_SHOULDER")): 
            patient_list = sum(1 for _ in study_category.iterdir())
            for patient_id in study_category.iterdir():
              if(os.path.isdir(patient_id)):
                for study_type in patient_id.iterdir():
                  for files in self.img_format_type:
                    filelist = list(study_type.glob('**/'+ files))
                  for fname in filelist:
                    class_label = self.label_img(fname)
                    for file in glob.glob(str(fname)):
                      resizedImg = self.image_read_resize(file,self.scaled_size)
                      self.x_list.append(resizedImg)
                      self.y_list.append(class_label)
                      x_npy = np.array(self.x_list)
                      y_npy = np.array(self.y_list)
      np.save(Path.joinpath(self.outpath,"Images_array"), x_npy)
      np.save(Path.joinpath(self.outpath,"Labels_array"), y_npy)
      return self.x_list, self.y_list, x_npy, y_npy

In [0]:
#Train dataset:
objImg = Image_Loader("/content/drive/My Drive/AIML/Capstone_Team/MURA-v1.1/","train",224,cv2.IMREAD_GRAYSCALE)

In [0]:
x_train, y_train, x_train_npy, y_train_npy  = objImg.image_to_list_numpy()

In [0]:
x_train_npy.shape, y_train_npy.shape

((8379, 224, 224), (8379,))

In [0]:
Images = np.load('/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/Processed_Image/XR_SHOULDER/train/Images_array.npy')
labels = np.load('/content/drive/My Drive/AIML/Capstone/MURA-v1.1/train/Processed_Image/XR_SHOULDER/train/Labels_array.npy')

In [0]:
Images.shape, labels.shape

((5, 224, 224), (5,))

In [0]:
#Validation/Test (original) dataset
objImg_test = Image_Loader("/content/drive/My Drive/AIML/Capstone_Team/MURA-v1.1/","valid",224,cv2.IMREAD_GRAYSCALE)

In [0]:
x_test, y_test, x_test_npy, y_test_npy  = objImg_test.image_to_list_numpy()

In [0]:
x_test_npy.shape, y_test_npy.shape 

((563, 224, 224), (563,))